## Define some functions
These functions:
1. Generate persistent blank node identifiers
2. Generate RDF patches

In [24]:
import uuid
from rdflib import Graph, Literal, URIRef, Namespace

# Namespace for your application
NAMESPACE = uuid.uuid5(uuid.NAMESPACE_URL, "https://graph-id")


def generate_address_uuid(customer_id, address_details):
    # Combine customer ID and address details to create a unique name
    name = f"{customer_id}:{address_details.get('street')}:{address_details.get('city')}:{address_details.get('state')}:{address_details.get('country')}"
    return uuid.uuid5(NAMESPACE, name)


def generate_rdf_ntriples(customer_id, address, uuid):
    """
    NB this function creates URIs for BlankNodes. 
    Jena understands URIs that start with _: as not being real URIs and instead handles these as blanknodes.
    This allows RDF patches to be created externally and refer to existing blank nodes within the graph - so long as they utilise the same identifier.
    The generate_address_uuid function is an example of how to achieve unique identifiers.
    """
    g = Graph()

    ex = Namespace("http://example.org/")

    addr = URIRef(f"_:{str(uuid)}")

    g.add((URIRef(f"http://example.org/customer/{customer_id}"), ex.hasAddress, addr))
    g.add((addr, ex.street, Literal(address.get('street'))))
    g.add((addr, ex.city, Literal(address.get('city'))))
    g.add((addr, ex.state, Literal(address.get('state'))))

    if address.get('country'):
        g.add((addr, ex.country, Literal(address.get('country'))))

    return g


def generate_patch(add_triples, delete_triples):
    patch = "TX .\n"

    for triple in delete_triples:
        patch += f"D {triple.strip()}\n"

    for triple in add_triples:
        patch += f"A {triple.strip()}\n"

    patch += "TC .\n"
    return patch

## Create initial RDF data
Start with some data from a relational source and convert to RDF, with persistent identifiers for Blank Nodes

In [25]:
customer_id = 'customer-123'
addresses = [
    {
        'street': '123 Main St',
        'city': 'Anytown',
        'state': 'CA'
    },
    {
        'street': '456 Elm St',
        'city': 'Othertown',
        'state': 'MN',
        'country': 'USA'
    }
]

# Generate UUIDs for the addresses
uuids = [generate_address_uuid(customer_id, address) for address in addresses]

# Generate RDF N-Triples for the addresses
add_triples_gs = [generate_rdf_ntriples(customer_id, address, uuid) for address, uuid in zip(addresses, uuids)]
g = Graph()
for address_g in add_triples_gs:
    g += address_g

triples = g.serialize(format="ntriples")
print(triples)

<_:d562967b-2c53-5864-9b92-d842d1bad2e6> <http://example.org/street> "123 Main St" .
<_:a69cf8fd-977c-545b-93b5-ba2a6ea05bfb> <http://example.org/state> "MN" .
<_:a69cf8fd-977c-545b-93b5-ba2a6ea05bfb> <http://example.org/country> "USA" .
<http://example.org/customer/customer-123> <http://example.org/hasAddress> <_:d562967b-2c53-5864-9b92-d842d1bad2e6> .
<http://example.org/customer/customer-123> <http://example.org/hasAddress> <_:a69cf8fd-977c-545b-93b5-ba2a6ea05bfb> .
<_:d562967b-2c53-5864-9b92-d842d1bad2e6> <http://example.org/state> "CA" .
<_:a69cf8fd-977c-545b-93b5-ba2a6ea05bfb> <http://example.org/street> "456 Elm St" .
<_:d562967b-2c53-5864-9b92-d842d1bad2e6> <http://example.org/city> "Anytown" .
<_:a69cf8fd-977c-545b-93b5-ba2a6ea05bfb> <http://example.org/city> "Othertown" .



## Update the data
1. Create a patch which updates one of the above addresses
2. Apply the patch (external to Jupyter)

NB: The above RDF would be created as an addition patch and sent to the RDF Delta server. For the purposes of this tutorial it can be persisted to disk, and the update patch that is created below can be applied to it.

In [26]:
# Example update for address 1
old_address = addresses[0]
new_address = {
    'street': '789 Oak St',
    'city': 'Anytown',
    'state': 'CA',
    'country': 'USA'
}
old_uuid = uuids[0]
new_uuid = generate_address_uuid(customer_id, new_address)

# Generate RDF N-Triples for the old and new addresses
delete_triples = generate_rdf_ntriples(customer_id, old_address, old_uuid).serialize(format="ntriples").splitlines()
new_add_triples = generate_rdf_ntriples(customer_id, new_address, new_uuid).serialize(format="ntriples").splitlines()

# Generate the patch
patch = generate_patch(new_add_triples, delete_triples)

print(patch)

TX .
D <_:d562967b-2c53-5864-9b92-d842d1bad2e6> <http://example.org/city> "Anytown" .
D <_:d562967b-2c53-5864-9b92-d842d1bad2e6> <http://example.org/street> "123 Main St" .
D <http://example.org/customer/customer-123> <http://example.org/hasAddress> <_:d562967b-2c53-5864-9b92-d842d1bad2e6> .
D <_:d562967b-2c53-5864-9b92-d842d1bad2e6> <http://example.org/state> "CA" .
A <_:79014a1e-ed41-5ae1-a6cf-017d4d7088ad> <http://example.org/street> "789 Oak St" .
A <_:79014a1e-ed41-5ae1-a6cf-017d4d7088ad> <http://example.org/country> "USA" .
A <_:79014a1e-ed41-5ae1-a6cf-017d4d7088ad> <http://example.org/city> "Anytown" .
A <http://example.org/customer/customer-123> <http://example.org/hasAddress> <_:79014a1e-ed41-5ae1-a6cf-017d4d7088ad> .
A <_:79014a1e-ed41-5ae1-a6cf-017d4d7088ad> <http://example.org/state> "CA" .
TC .



## Test with RDF Delta

1. Download RDF Delta from: https://repo1.maven.org/maven2/org/seaborne/rdf-delta/rdf-delta-dist/
2. Save the above ntriples file as "addresses.nt".  
3. Save the above update patch as "update.rdp".  

With the Jena command line tools installed you can then run:

```sh
dcmd p2r --data addresses.nt update.rdp > updated_addresses.ttl
```

This produces the following output:
NB the "Anytown" address has been updated.

```turtle
<http://example.org/customer/customer-123>
        <http://example.org/hasAddress>
                [ <http://example.org/city>     "Othertown" ;
                  <http://example.org/country>  "USA" ;
                  <http://example.org/state>    "MN" ;
                  <http://example.org/street>   "456 Elm St"
                ] ;
        <http://example.org/hasAddress>
                [ <http://example.org/city>     "Anytown" ;
                  <http://example.org/country>  "USA" ;
                  <http://example.org/state>    "CA" ;
                  <http://example.org/street>   "789 Oak St"
                ] .
```